In [35]:
# install below dependencies
# !pip install gymnasium
# !pip install pygame
# !pip install stable-baselines3


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os

from typing import Dict, Any, Optional, Callable

import gymnasium as gym
from gymnasium.envs.registration import register, registry
from gymnasium.wrappers import RecordVideo
import time
import numpy as np

# from typing import Any, Dict
import torch
import torch.nn as nn

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

2025-02-13 18:17:23.817982: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 18:17:23.832797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739470643.852466    4279 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739470643.862167    4279 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 18:17:23.911208: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Deep Reinforcement Learning for COLREG-Compliant Autonomous Surface Vehicle Simulation
*Project by Stanislav Dobrev for the SoftUni Deep Learning Course*

## Abstract

Maritime transportation plays a critical role in global trade, accounting for over 80% of the world's goods movement. As automation advances, the development of autonomous surface vehicles (ASVs) is becoming increasingly relevant for safer and more efficient navigation. However, ensuring that ASVs comply with the International Regulations for Preventing Collisions at Sea (COLREGs) remains a significant challenge.

This study presents a partially implemented COLREG-compliant ASV simulation, where a reinforcement learning (RL) agent, trained using Proximal Policy Optimization (PPO), is responsible for navigating toward a waypoint while avoiding potential collisions. The agent is designed to handle static obstacles, head-on encounters, and crossing targets in a dynamic environment. Unlike many previous approaches that incorporate explicit waypoint tracking algorithms or cross-track error corrections, this implementation relies solely on a reward-based learning mechanism to shape the agent’s decision-making process.

By leveraging a structured reward function, the ASV learns collision avoidance behaviors without predefined path-following rules, demonstrating an emergent compliance with fundamental COLREG principles. The study provides insights into the effectiveness of reinforcement learning in maritime navigation and highlights the potential and limitations of a purely reward-driven training paradigm. Further improvements and extensions, including enhanced optimization strategies and real-world applicability, are discussed as part of future work.

## 1. Introduction and Problem Definition
### 1.1 The Future of Autonomous Ships
The maritime industry is undergoing a transformative shift with the advent of autonomous ships, driven by advances in artificial intelligence, automation, and reinforcement learning. The potential benefits of autonomous surface vehicles (ASVs) include increased efficiency, reduced operational costs, and enhanced safety by minimizing human errors. Major stakeholders, including research institutions, shipping companies, and regulatory bodies, are actively exploring autonomous navigation solutions. However, the implementation of fully autonomous ships remains in its early stages, facing numerous challenges such as regulatory compliance, collision avoidance, and real-world operational feasibility.

Current research on ASVs has made significant strides in integrating reinforcement learning (RL) techniques to develop autonomous navigation systems that can react to dynamic maritime environments. However, achieving full compliance with the International Regulations for Preventing Collisions at Sea (COLREGs) remains a critical hurdle. Most RL-based approaches focus on collision avoidance but often lack explicit mechanisms for adhering to navigation rules, especially in complex multi-vessel scenarios. This research aims to partially implement COLREG-compliant behavior in an RL-trained ASV, focusing specifically on key collision avoidance rules.

### 1.2 The Role of the Human Factor in Marine Navigation
Traditional marine navigation relies heavily on human judgment and experience to assess and mitigate collision risks. Despite technological advancements such as radar, AIS (Automatic Identification System), and electronic chart systems, human errors remain a leading cause of maritime accidents. Fatigue, misinterpretation of navigational data, and delayed decision-making contribute significantly to collision incidents. Autonomous navigation systems have the potential to mitigate these risks by ensuring more consistent, data-driven decision-making. However, for such systems to be viable, they must adhere to the same legal and operational standards as human-operated vessels.

Ensuring compliance with COLREGs is a crucial aspect of integrating ASVs into real-world maritime traffic. While human mariners rely on experience and situational awareness to make decisions, an RL-based ASV must learn collision avoidance through a structured reward system. This study explores how an ASV can navigate toward a waypoint while avoiding collisions by learning COLREG-compliant maneuvers without explicit waypoint-tracking algorithms.

### 1.3 Definition and Key COLREG Rules
The International Regulations for Preventing Collisions at Sea (COLREGs) are the globally accepted rules that govern vessel movements to prevent collisions. These rules apply to all vessels at sea and dictate how ships should interact in different navigation scenarios. The following COLREG rules are particularly relevant to this study:

**Rule 14: Head-on Situation**
When two power-driven vessels are approaching on reciprocal or nearly reciprocal courses, both vessels must alter course to starboard to ensure they pass on each other’s port side.

**Rule 15: Crossing Situation**
When two power-driven vessels are crossing and there is a risk of collision, the vessel that has the other on its starboard side must keep out of the way and, if possible, avoid crossing ahead of the other vessel.

**Rule 16: Action by the Give-Way Vessel**
Any vessel required to keep out of the way of another vessel must take early and substantial action to stay well clear.

**Rule 6: Safe Speed**
Every vessel must proceed at a speed that allows it to take proper and effective action to avoid collisions, considering the prevailing circumstances and conditions.

**Rule 8: Action to Avoid Collision:**

- (b) Course or speed alterations must be substantial enough to be readily apparent to other vessels. A series of small alterations should be avoided.
- (c) If there is enough sea room, altering course alone may be the most effective way to avoid a close-quarters situation, provided it is done in good time and does not create another hazard.
- (d) The action taken must ensure a safe passing distance and should be continuously monitored until the vessels are past and clear.
- (e) If necessary, a vessel should reduce speed or stop to assess the situation better and avoid collision.


The following COLREG rules are omitted in this research:

**Rule 17: Action by the Stand-on Vessel** – The responsibilities of the vessel that has the right of way are not considered in this study.

**Rule 13: Overtaking** – Situations involving overtaking maneuvers are not included due to the complexity of modeling them within the given scope.

For the purpose of the study, own ship is **ALWAYS GIVE-WAY** vessel and she must act according to the rules.

### 1.4 Assumptions and Limitations
This study makes several simplifying assumptions to focus on reinforcement learning-based collision avoidance without additional complexity:

- Ship dynamics are ignored – The ASV's maneuvering characteristics, such as acceleration, turning radius, and inertia, are not explicitly modeled. The vessel is assumed to change course and speed instantly without delay.
- Environmental dynamics are ignored – External factors like wind, currents, and waves are not considered in the simulation. The focus is solely on collision avoidance in an idealized static environment.
- Target dynamics are stripped to basics - maintaining course and speed.
- Limited scope due to time and complexity – Implementing full COLREG compliance in an RL-based agent is highly complex and time-consuming. Therefore, this research prioritizes the fundamental rules of collision avoidance and waypoint navigation.


By defining these constraints, this study aims to analyze how reinforcement learning alone, without traditional waypoint-tracking algorithms or cross-track error corrections, can develop COLREG-compliant behavior in an ASV. The results provide a foundation for future improvements and real-world applications.

## 2. Previous Research
Several research studies have explored the integration of reinforcement learning (RL) into autonomous surface vehicles (ASVs) to ensure compliance with the International Regulations for Preventing Collisions at Sea (COLREGs). Notable contributions include:

**[Risk-Based Implementation of COLREGs Using Deep Reinforcement Learning](https://arxiv.org/abs/2112.00115)**: This study presents a method that incorporates a subset of COLREGs into a deep RL-based path-following and obstacle avoidance system. The approach dynamically balances path following and COLREG-compliant collision avoidance in various scenarios, including training environments and real-world simulations.

**[COLREGs-Compliant Autonomous Collision Avoidance Method](https://arxiv.org/abs/2006.09540)**: This research introduces an algorithm that enables unmanned surface vehicles (USVs) to navigate safely to target points without collisions. The proposed method ensures that decisions made by the USV adhere to COLREGs, enhancing maritime safety. 

**[Collision Avoidance for Autonomous Ships Using Deep Reinforcement Learning](https://www.frontiersin.org/journals/marine-science/articles/10.3389/fmars.2022.1084763/full)**: The study develops an intelligent collision avoidance algorithm based on approximate representation reinforcement learning (AR-RL). By combining prior knowledge with COLREGs, the model optimizes performance in dynamic environments, facilitating safe navigation for maritime autonomous surface ships (MASS). 

**[COLREGs-Based Path Planning for USVs Using Deep Reinforcement Learning](https://www.mdpi.com/2077-1312/11/12/2334)**: This research proposes a two-stage deep reinforcement learning approach for cooperative path planning of unmanned surface vehicles. The method addresses collision avoidance while adhering to COLREGs and considers both intra-fleet and external vessel interactions. 

**[A Novel Reinforcement Learning Collision Avoidance Algorithm for USVs](https://www.researchgate.net/publication/359110601_A_Novel_Reinforcement_Learning_Collision_Avoidance_Algorithm_for_USVs_Based_on_Maneuvering_Characteristics_and_COLREGs)**: This paper presents an autonomous collision avoidance algorithm that complies with USV maneuverability and COLREGs. The reinforcement learning agent learns collision avoidance maneuvers without prior human knowledge, utilizing a double-DQN method to reduce overestimation of the action-value function.

## 3. Environment
The simulation environment is implemented using Gymnasium, a framework widely used for reinforcement learning (RL). The MarineEnv class inherits from gym.Env, making it compatible with reinforcement learning libraries such as Stable-Baselines3. By extending gym.Env, the environment adheres to the standard structure required for RL training, including predefined methods such as `reset(), step(), render(), and close()`. This allows seamless interaction with RL agents, which learn optimal navigation strategies through trial and error.

### 3.1 Observation Space
The observation space in this environment is continuous, meaning it consists of real-valued numerical inputs that describe the agent’s current state. The observation space provides information on the own ship, next waypoint and surrounding targets , allowing the agent to make informed navigation decisions.

**Own Ship State:** The ASV's state includes its course, speed, distance to the waypoint (WP), estimated time of arrival (ETA), and relative bearing to the WP. This data helps the agent assess progress toward the navigation goal.

**Target Ships State:** Each detected target ship is represented by its bearing, closest point of approach (CPA), time to CPA (TCPA), course, speed, and distance. These parameters are crucial for collision avoidance.


The observation space is structured using a method called `_define_observation_space()`, which ensures consistency and scalability when adding more features or refining the environment. The environment supports multiple targets, each represented within a tuple of dictionaries. Targets are dynamically assigned one of three possible encounter situations: crossing, head-on, or static.

The geographical bounds of the environment are computed based on the `ENV_RANGE` parameter, which determines the navigable area in nautical miles. The ASV starts at `(0.0, 0.0)`, and all movements are calculated relative to this initial reference point.

Scaling of the environment is set using `timescale` parameter with a default value of 1, meaning each step in the environment equals 1 minute of real-world time.

### 3.2 Action Space
The environment employs a continuous action space, allowing the RL agent to make fine-grained adjustments to the ASV’s movement. The agent controls two variables:

**Course Change:** The agent can adjust the ship’s heading within a range of `-1` (maximum turn to port) to `1` (maximum turn to starboard). The maximum turning rate is 20 degrees per minute, meaning a full `1` value corresponds to a 20-degree starboard turn.

**Speed Change:** The agent can accelerate or decelerate within a range of `-1` to `1`, where `1` corresponds to an increase of 0.5 knots per minute and `-1` results in a decrease of 0.5 knots per minute.

Each simulation step represents one minute of real-world time. This means that a continuous change in course and speed accumulates over multiple time steps, reflecting realistic ship maneuvering constraints.

During training, the environment operates with a timescale of 1/3, meaning each step represents 20 seconds of real-world time. All course and speed adjustments, as well as position updates, are scaled accordingly to reflect this timescale.

### 3.3 Reset Method
The `reset()` method initializes the environment at the start of an episode. It sets up the own ship, assigns a waypoint, and places target ships in realistic maritime scenarios.

#### 3.3.1 Waypoint Setup
The waypoint (WP) serves as the navigation goal for the ASV. It is randomly placed within a predefined range from the initial position to ensure variability across episodes. The WP’s distance is constrained to be within a practical navigation range, ensuring that the ASV has enough maneuvering room to encounter target ships along the way.

When the WP is placed, the ASV calculates:

- Distance to WP – How far the ship must travel to reach the target.
- ETA to WP – The estimated arrival time based on current speed.
- Relative Bearing to WP – The directional offset between the ASV’s current heading and the WP’s position.

#### 3.3.2 Target Setup
Target ships are introduced into the environment based on different encounter scenarios:

- Crossing Situation: A target approaches from the starboard side, requiring the ASV to act as a give-way vessel by altering to starboard or/and reducing speed.
- Head-on Situation: The ASV and target vessel are moving toward each other on reciprocal courses, necessitating a starboard turn per COLREG Rule 14.
- Static Target: A stationary obstacle that the ASV must avoid.

Each target is assigned a course, speed, and initial position based on real-world maritime navigation principles. The placement ensures that at least one target represents a collision risk during the episode.

### 3.4 Step Method
The `step()` method simulates the own ship’s movement and updates the state of detected targets. The RL agent selects an action, which affects the ASV’s course and speed.

#### 3.4.1 Own Ship and Target Movement
The own ship updates its course and speed based on the selected action.
Detected target ships move according to their assigned speed and course, dynamically updating their CPA and TCPA relative to the ASV.
If a target moves beyond the simulation bounds, it is removed from the active target list.
#### 3.4.2 Reward Function
The reward function evaluates the agent’s actions based on two primary objectives:

**Waypoint Tracking**: If no immediate collision threats are detected, the agent receives rewards based on progress toward the waypoint. Positive rewards are given for:

- Reducing the distance to the WP.
- Maintaining alignment with the WP’s bearing (ASV stays on track).
- Maintaining ETA to the WP (No random speed changes), 

**COLREG-Compliant Collision Avoidance**: If a target is classified as dangerous (i.e., it falls within CPA and TCPA limits), COLREG rules take precedence over waypoint tracking. During this phase:

- Waypoint tracking is put on hold until the ASV successfully avoids the dangerous target.
- The agent is rewarded for correct COLREG-compliant maneuvers, such as making a starboard turn in a head-on encounter or giving way in a crossing situation.
- Penalties are applied for incorrect actions, such as violating Rule 14 (head-on) or making unnecessary maneuvers.
- A collision penalty is applied if the ASV fails to avoid an obstacle. If the ASV successfully passes a target at a safe CPA distance, it resumes WP tracking and continues navigation toward its goal.

### 3.5 Ship Classes and Methods
The simulation includes three types of vessels:

#### 3.5.1 Base Ship
The BaseShip class defines core properties and movement methods applicable to all vessels:

- Position (latitude, longitude)
- Course (heading in degrees)
- Speed (knots)

Basic navigation methods for calculating bearing, distance, and speed adjustments.

#### 3.5.2 Own Ship
The OwnShip class extends BaseShip and introduces:

- Target Detection: Identifies nearby vessels and classifies them as dangerous or safe.
- Relative Navigation: Calculates CPA and TCPA for collision avoidance.

The class incorporates methods for target/wp tracking.

#### 3.5.3 Target Ship
The Target class extends BaseShip and includes:

- Relative Bearing and Distance to the ASV
- Aspect Classification (Crossing, Head-on, or Static)
- Danger Status (Determines if collision avoidance should be applied)

### 3.6 Video Rendering and Visualization
The environment includes real-time rendering using Pygame, allowing for visual monitoring of the agent’s training process. The graphical representation follows a structured color scheme to distinguish between different vessels, waypoints, and motion vectors.

#### 3.6.1 Visualization Elements
- White dot → Represents the own ship (ASV).
- Green dot → Represents the waypoint (WP), the ASV’s navigation goal.
- Blue dots → Represent the target ships, which may be moving or static obstacles.
#### 3.6.2 Motion Vectors
To provide an intuitive representation of ship movements:

- Red line → Represents the own ship’s heading vector, with its length scaled to the ASV’s speed in knots.
- Blue line → Represents the heading vector of target ships, similarly scaled to target speed.
#### 3.6.3 Tracking and CPA Visualization
- Purple ring → Displays the Closest Point of Approach (CPA) limit set for collision avoidance (default: 1 NM).
- White track line → Shows the historical path (trace) of the own ship, allowing for trajectory analysis.
- Blue track lines → Represent the path history of target ships, illustrating their movement over time.
#### 3.6.4 Rendering in Training and Evaluation
During training, rendering is optional and mainly used for debugging.

In evaluation runs, rendering helps assess the agent’s COLREG compliance, decision-making, and waypoint tracking performance.
This visualization provides an intuitive representation of maritime navigation within the RL simulation, making it easier to analyze how the ASV interacts with waypoints and target vessels in different scenarios.

By integrating these components, the simulation environment provides a realistic and structured testbed for training an RL-based ASV, ensuring that it navigates effectively while complying with maritime collision regulations.

## 4. Model Training and Selection
The training process for the COLREG-compliant ASV is structured into several stages, involving model selection, hyperparameter tuning, and evaluation. The goal is to develop an RL agent capable of waypoint navigation and collision avoidance, ensuring compliance with COLREGs.

### 4.1 Model Selection: Why DRL? Why PPO?

#### 4.1.1 Why Reinforcement Learning Instead of Standard Machine Learning?
Traditional supervised learning approaches in machine learning rely on large labeled datasets to map inputs to outputs. However, maritime navigation is an inherently sequential decision-making problem, where an agent must continuously adjust its actions based on dynamic environmental conditions.

Using standard machine learning techniques for ship navigation would be impractical due to:

- Lack of labeled data: The model would require manually labeled examples of every possible scenario, which is infeasible for complex maritime interactions.
- Sequential dependencies: Maritime navigation involves long-term decision-making—an action taken now affects the future state of the vessel.
- Exploration-exploitation tradeoff: Standard ML models do not explore alternative strategies, whereas reinforcement learning (RL) allows an agent to discover optimal maneuvers over time.

Deep RL, enables an Autonomous Surface Vehicle (ASV) to learn by interacting with its environment and improving its policy iteratively. Deep RL is more flexible than traditional ML because it:

- Learns optimal policies without explicit supervision.
- Adapts to dynamic environments (e.g., moving targets, changing encounter scenarios).
- Can handle continuous control problems, such as adjusting course and speed in real-time.

Thus, deep reinforcement learning is a natural fit for COLREG-compliant ASV navigation, as it balances waypoint tracking and collision avoidance dynamically.

#### 4.1.2 Why Proximal Policy Optimization (PPO)?
PPO was chosen as the reinforcement learning framework for this project due to its stability, sample efficiency, and suitability for continuous control tasks. PPO falls under Policy Gradient Methods, where the policy (agent's behavior) is directly optimized.

Key Advantages of PPO for Maritime Navigation
- Stability and Sample Efficiency

PPO introduces a clipped objective function, which prevents large, unstable policy updates.
This improves training stability, avoiding drastic behavior shifts that could lead to unsafe ASV maneuvers.
Unlike traditional Policy Gradient methods, PPO reduces variance without requiring second-order derivatives (like Trust Region Policy Optimization, TRPO). Not too sensitive to hyperparameters tunning (as seen from the optimization results)
- Continuous Action Space Handling

ASVs require precise adjustments in heading and speed to comply with COLREG rules.
PPO operates well in continuous action spaces, where minor corrections are crucial for safe navigation.
Other RL methods (e.g., DQN) struggle with continuous control problems.
- Scalability for Parallel Training

PPO allows training across multiple parallel environments (n_envs=8 in this setup).
This accelerates learning by gathering diverse experiences in different maritime scenarios.
More training data leads to faster convergence.
- Robustness in Dynamic Environments

The maritime environment is highly dynamic, with moving target vessels.
PPO adapts well to changing conditions, learning from a variety of COLREG-compliant encounters.
The policy-based approach enables PPO to generalize to new situations better than pure value-based methods.

In this project, PPO with Clipped Surrogate Objective is used. This means the model optimizes the objective function by taking the minimum of the unclipped objective and the clipped surrogate objective, preventing excessively large policy updates to improve stability. 

The clipped loss function is the expected value over time steps of the minimum between two terms: first, the probability ratio, which measures the change in policy probability between the new and old policy, multiplied by the advantage estimate, which indicates how much better an action is compared to the expected return. Second, the clipped probability ratio, which restricts the change within a specified range to prevent overly large policy updates, also multiplied by the advantage estimate.

$$
L_{\text{CLIP}}(\theta) = \hat{\mathbb{E}}_t \left[ \min \left( r_t(\theta) \hat{A}_t, \, \text{clip} \left( r_t(\theta), 1 - \epsilon, 1 + \epsilon \right) \hat{A}_t \right) \right]
$$
where:
- $r_t(\theta) = \frac{\pi_{\theta}(a_t \mid s_t)}{\pi_{\theta_{\text{old}}}(a_t \mid s_t)}$ is the ratio between the new policy and the old policy
- $\hat{A}_t$ is the advantage estimation
- $\hat{\mathbb{E}}_t$ is the expectation over timesteps,
- $\epsilon$ is the clip coefficient

The final loss function in Proximal Policy Optimization (PPO) is a weighted combination of three components - policy loss, value loss and entropy loss:

$$
L_{\text{CLIP+VF+S}}(\theta) = \hat{\mathbb{E}}_t \left[ L_{\text{CLIP},t}(\theta) - c_1 L_{\text{VF},t}(\theta) + c_2 S[\pi_{\theta}](s_t) \right]
$$

where:
- $L_{\text{CLIP+VF+S}}(\theta)$ is the combined total loss,
- $L_{\text{CLIP},t}(\theta)$ is the clipped policy loss,
- $c_1 L_{\text{VF},t}(\theta)$ is the value function loss, multiplied by a weighting coefficient $c_1$,
- $c_2 S[\pi_{\theta}](s_t)$ is the entropy loss, multiplied by a weighting coefficient $c_2$.

In [2]:
# Create folders for logs and videos
log_folder = './project_logs'
video_folder = os.path.join(log_folder, 'video')
os.makedirs(video_folder, exist_ok=True)
PPO.load(os.path.join(log_folder, 'optuna/best_model/best_model.zip'), device='cpu').policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=36, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=36, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=2, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

#### 4.1.3 Policy Architecture Overview
The Actor-Critic Policy used in the PPO agent consists of several key components responsible for feature extraction, policy learning, and value estimation. Below is an overview of the architecture:

**Feature Extractors**

The model includes three FlattenExtractor modules:

- General feature extractor for processing raw observations.
- Policy feature extractor $\pi$ to transform state inputs for action selection.
- Value function feature extractor $\text{V}$ to estimate state values for advantage computation.
These extractors flatten input tensors to make them suitable for processing by the Multi-Layer Perceptron (MLP) network.

**MLP Feature Extractor**

The core of the network is the MlpExtractor, which consists of two independent networks:

- Policy Network: Maps input features to action probabilities.
- Value Network: Maps input features to a scalar value representing expected returns from a given state.

Each network has:

- Two fully connected layers with 64 neurons each.
- Tanh activation functions ensuring smooth, bounded outputs.

**Output Layers**

- Action Network: A linear layer mapping the last policy layer (64 features) to 2 output neurons, representing the action space.
- Value Network: A linear layer mapping the last value layer (64 features) to a single scalar, estimating the value of a given state.

This architecture ensures that the actor (policy network) and critic (value network) share representations while maintaining separate pathways for action selection and value estimation, following the actor-critic reinforcement learning paradigm.

In [3]:
%matplotlib inline
%load_ext tensorboard

%tensorboard --logdir {log_folder} --host=0.0.0.0

In [4]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='environments:MarineEnv',  # String reference to the class
    )

def yield_random_seed():
    while True:
        yield np.random.randint(low=1, high=201)
seed_generator = yield_random_seed()

### 4.2 Training Process

Tensorboard was used to visualize the learning progress of the agent.

#### 4.2.1 Initial Training with Default Hyperparameters
The training process began using default PPO hyperparameters to establish a baseline performance. At this stage, the RL agent was trained in a controlled environment where the ASV learned to:

- Move toward a waypoint.
- Avoid static obstacles.
- Perform simple navigation maneuvers.

The default parameters used in initial training included:

- gamma = 0.99 (discount factor for long-term rewards)
- clip_range = 0.2 (limits the size of policy updates)
- n_steps = 2048 (number of steps before updating the policy)
- batch_size = 64 (batch size for training)
- gae_lambda = 0.95 (Generalized Advantage Estimation smoothing)

This initial phase provided insight into the agent’s basic behavior and highlighted areas that required improvement.

#### 4.2.2 Hyperparameter Optimization with Optuna
To improve performance, an extensive hyperparameter tuning process was conducted using Optuna, an optimization framework that automates the search for optimal hyperparameters. The key objectives were:

- Improving COLREG compliance: Ensuring that the ASV consistently follows maritime collision avoidance rules.
- Balancing waypoint tracking and collision avoidance: Ensuring that the ASV prioritizes waypoint tracking when no immediate collision risk is present.
- Optimizing learning stability and efficiency: Preventing issues like premature convergence or overly cautious navigation.

The optimization of the model will be further explained in Section 5 and new model will be trained with best hyperparameters according to the Optuna study.

In [5]:
# setting the environment kwargs
timescale = 1 / 3
env_kwargs = dict(
    render_mode='rgb_array',
    continuous=True,
    max_episode_steps=int(400 / timescale),
    training_stage=2,
    timescale=timescale,
    training=True,
    total_targets=3,
)

### 4.3 Training Execution
#### 4.3.1 Multi-Environment Training Setup
The agent was trained using 8 parallel environments (n_envs=8) to accelerate learning. This allowed the model to gather experience faster, reducing training time while improving generalization.

A separate evaluation environment was created to periodically assess the agent’s performance without affecting training. This evaluation was performed every 5000 steps, and the best-performing model was saved automatically.

Video recording was performed during training. Videofiles located at `./project_logs/videos/training`

In [6]:
# setup of the training envs
train_env = make_vec_env(env_id='MarineEnv-v0', n_envs=8, env_kwargs=env_kwargs)
eval_env = gym.make('MarineEnv-v0', **env_kwargs)
eval_env = Monitor(eval_env)

# video recording of the training process
# eval_env = RecordVideo(eval_env, video_folder, episode_trigger=lambda x: x % 2 == 0)

In [7]:
# eval callback - used to track the statistics of the training
eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=os.path.join(log_folder, 'default/best_model/'),
        log_path=os.path.join(log_folder, 'default/results/'),
        eval_freq=5000,
        deterministic=True,
        render=False
    )

In [8]:
# setup of the defaut kwargs for initial training of PPO agent
default_kwargs = {
    'learning_rate': 3e-4,
    'n_steps': 2048,
    'batch_size': 64,
    'n_epochs': 10,
    'gamma': 0.99, 
    'gae_lambda': 0.95,
    'clip_range': 0.2,
    'ent_coef': 0.0,
    'vf_coef': 0.5, 
    'max_grad_norm': 0.5,
    'target_kl': None,
    'tensorboard_log': log_folder,
}

In [9]:
# establish initial agent
agent = PPO(
        policy='MlpPolicy',
        env=train_env,
        verbose=0,
        device='cpu',
        **default_kwargs,
    )

The agent was then trained for 600,000 timesteps with a progress bar for real-time monitoring.

In [10]:
# agent.learn(total_timesteps=int(6e5), reset_num_timesteps=True, tb_log_name='default/default_best', progress_bar=True, callback=eval_callback)

#### 4.3.2 Model Evaluation
After training, the model was evaluated using 10 test episodes, where it was deployed in a separate environment `(eval_env)` with identical parameters.

The performance was measured using mean and standard deviation of rewards:

In [11]:
default_best_agent = PPO.load(os.path.join(log_folder, 'default/best_model/best_model.zip'), device='cpu')
eval_env = Monitor(gym.make('MarineEnv-v0', **env_kwargs))
mean, std = evaluate_policy(model=default_best_agent, env=eval_env, n_eval_episodes=10, deterministic=True)
print(f'Mean: {mean:.2f}, Std: {std:.2f}')

Mean: 564.86, Std: 384.15


### 4.4 Model Deployment and Visualization
#### 4.4.1 Testing the Trained Model
The trained agent was deployed in a human-rendered environment (render_mode='human'), where it performed real-time decision-making in a scenario with 3 dynamic target ships.

Each episode ran for up to 400 time steps, simulating more than 50 minutes of real-world navigation per episode.

During each test episode, ASV’s step-by-step reward accumulation was logged. This real-world simulation provided qualitative validation of the model's COLREG compliance and maneuvering decisions.

In [12]:
def run_video_rendering(
    agent: Any, 
    episodes: int = 3, 
    timescale: float = 1/6, 
    seed: Optional[int] = None, 
    record_video: bool = False, 
    episode_trigger: Optional[Callable[[int], bool]] = None, 
    name_prefix: Optional[str] = None, 
    video_folder: Optional[str] = None
) -> None:
    """
    Runs a simulation of the MarineEnv-v0 environment using the given agent, with optional video recording.

    Parameters:
    - agent: The trained agent used for inference.
    - episodes (int): Number of episodes to run (default is 3).
    - timescale (float): The simulation timescale factor (default is 1/6).
    - seed (int, optional): Random seed for environment initialization. If None, a generated seed is used.
    - record_video (bool): If True, records the simulation as a video (default is False).
    - episode_trigger (function, optional): A function that determines which episodes get recorded.
    - name_prefix (str, optional): Prefix for recorded video file names.
    - video_folder (str, optional): Path to save recorded videos.

    Behavior:
    - If video recording is enabled, the environment is wrapped with RecordVideo.
    - Logs episode statistics, including total rewards, episode length, and termination status.
    - If recording, logs are saved to a text file in the specified video folder.
    - Displays simulation results in the console if video recording is disabled.

    Returns:
    - None. The function either logs the results to a file or prints them to the console.
    """
    
    if seed is None:
        seed = next(seed_generator)
    
    env = gym.make(
        'MarineEnv-v0',
        render_mode='rgb_array' if record_video else 'human',
        continuous=True,
        training_stage=2,
        timescale=timescale,
        training=False,
        total_targets=3,
        seed=seed,
    )
    
    if record_video:
        from IPython.display import HTML
        from base64 import b64encode
        
        if episode_trigger is None:
            episode_trigger = lambda episode_id: True

        if video_folder is None:
            video_folder = video_folder   
        else:
            if name_prefix:
                video_folder = os.path.join(video_folder, name_prefix)
        
        # wrap environment for video recording
        env = RecordVideo(
            env=env, 
            video_folder=video_folder, 
            episode_trigger=lambda episode_id: True, 
            name_prefix=name_prefix)
        
    logged_episodes = []
    logged_rewards = []
    for episode in range(episodes):
        state, _ = env.reset()
        episode_rewards = 0
        eta = state[5]
        
        for step in range(int(400 / timescale)):
            with torch.no_grad():
                action = agent.predict(state, deterministic=True)
            
            state, reward, terminated, truncated, info = env.step(action[0])
            
            if not record_video:
                env.render()
                
            episode_rewards += reward
            
            if terminated or truncated:
                break
                
            time.sleep(0.005)
         
        result_string = f'Episode: {episode}\nEpisode length: {step}, Elapsed real time: {round(step * timescale)} minutes, Initial WP ETA: {round(eta)} minutes\nEpisode total rewards: {episode_rewards :.2f}\nIs terminated: {info["terminated"]}, Is truncated: {info["truncated"]}\n============================\n'

        logged_episodes.append(result_string)
        logged_rewards.append(episode_rewards)

    evaluation = f'Mean: {np.array(logged_rewards).mean():.2f}, Std: {np.array(logged_rewards).std():.2f}, Initial seed: {seed}'
    logged_episodes.append(evaluation)
    # Open log file
    if record_video:
        log_file_path = os.path.join(video_folder, name_prefix + '.txt')
        with open(log_file_path, 'w') as log_file:
            log_file.write('\n'.join(logged_episodes))
            print(f"Training log saved at: {log_file_path}")
    else:
        print('\n'.join(logged_episodes))
    
    env.close()

In [13]:
run_video_rendering(default_best_agent, episodes=3)

Seed set to 152
Episode: 0
Episode length: 302, Elapsed real time: 50 minutes, Initial WP ETA: 48 minutes
Episode total rewards: 368.58
Is terminated: WP Reached!, Is truncated: False

Episode: 1
Episode length: 360, Elapsed real time: 60 minutes, Initial WP ETA: 59 minutes
Episode total rewards: 754.62
Is terminated: WP Reached!, Is truncated: False

Episode: 2
Episode length: 567, Elapsed real time: 94 minutes, Initial WP ETA: 97 minutes
Episode total rewards: 741.98
Is terminated: WP Reached!, Is truncated: False

Mean: 621.72, Std: 179.07, Initial seed: 152


#### 4.4.2 Video Recording of Model Performance
To visually inspect the ASV’s behavior, episodes were recorded using Gymnasium’s RecordVideo wrapper. The video files were automatically saved and later reviewed to analyze decision-making behavior.

Multiple episodes were recorded to assess:

- The agent’s reaction to head-on, crossing, and static targets.
- Whether the ASV consistently made starboard turns in head-on encounters (Rule 14).
- How well the ASV balanced COLREG compliance with efficient waypoint tracking.

Recorded videos and episode statistics are provided in txt file, located:
```bash
./project_logs/video/defaut_agent/
./project_logs/video/optimized_agent/
```

### 4.5 Summary
The training process followed a structured pipeline:

1. Model Selection – PPO was chosen for its stability and suitability for continuous control.
2. Initial Training – Conducted with default PPO parameters to establish baseline performance.
3. Hyperparameter Optimization – Used Optuna to fine-tune mainly learning rate, discount factor, clipping range, and entropy coefficient.
4. Training Execution – Conducted using 8 parallel environments for efficiency, with regular evaluation checkpoints.
5. Evaluation and Testing – The model was tested across multiple episodes to measure waypoint tracking and COLREG compliance.
6. Deployment and Visualization – The trained model was deployed in a real-time simulation and video recordings were used for analysis.

Through this approach, the ASV learned safe, COLREG-compliant navigation strategies purely through reinforcement learning, without explicit rule-based waypoint tracking or cross-track error corrections.

## 5. Optimization
The optimization process aimed to fine-tune PPO hyperparameters for optimal performance in training a COLREG-compliant ASV (Autonomous Surface Vehicle). The primary goal was to balance waypoint tracking and collision avoidance while ensuring stable training. This was achieved using Optuna, an automated hyperparameter tuning framework that efficiently searches for the best configuration.

### 5.1 Optimization Strategy
To systematically optimize the reinforcement learning model, the following strategy was employed:

#### 5.1.1 Search Algorithm and Pruning Strategy
- Sampler: Tree-structured Parzen Estimator (TPESampler) was chosen as the search algorithm, which efficiently selects promising hyperparameters based on past trials.
- Pruner: A Median Pruner was used to terminate underperforming trials early, preventing unnecessary computations on bad configurations. This allowed the search process to focus computational resources on the most promising hyperparameter sets.

### 5.2 Hyperparameter Tuning Approach
#### 5.2.1 Optimization Setup

Following parameters used to run the study:

In [14]:
N_TRIALS = 100  # max number of trials
N_JOBS = 1 # number of jobs to run in parallel
N_STARTUP_TRIALS = 5  # stop random sampling after N_STARTUP_TRIALS
N_EVALUATIONS = 10  # number of evaluations during the training
N_TIMESTEPS = int(2e5)  # training budget
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 8 # number of the env for evaluation
N_EVAL_EPISODES = 10
# TIMEOUT = int(60 * 15)  # 15 minutes
TIMEOUT = None # keep going till finish

ENV_ID = 'MarineEnv-v0'

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
}

The **evaluation metric** used was the **mean episodic reward**, which reflects how well the agent balances waypoint tracking and collision avoidance.

#### 5.2.2 Hyperparameters Being Optimized
Each trial used random sampling at the beginning (N_STARTUP_TRIALS), followed by Bayesian optimization via the TPE algorithm.

Following the hyperparameters being optimized:

In [15]:
import optuna

def sample_ppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    Samples hyperparameters for Proximal Policy Optimization (PPO) using Optuna.

    Parameters:
    - trial (optuna.Trial): The Optuna trial object used for suggesting hyperparameter values.

    Returns:
    - Dict[str, Any]: A dictionary containing the sampled hyperparameters for PPO.
    
    Sampled Hyperparameters:
    - learning_rate (float): Learning rate for policy optimization, sampled logarithmically between 5e-5 and 1e-3.
    - n_steps (int): Number of environment steps per update, sampled as a power of 2 between 1024 and 4096.
    - batch_size (int): Batch size chosen from valid divisors of (n_steps * N_EVAL_ENVS).
    - n_epochs (int): Number of PPO epochs per batch update, sampled between 8 and 12.
    - gamma (float): Discount factor for future rewards, sampled between 0.95 and 0.9999.
    - gae_lambda (float): Generalized Advantage Estimation (GAE) lambda, balancing bias-variance trade-off, sampled between 0.9 and 1.0.
    - clip_range (float): Clipping range for PPO updates, sampled between 0.15 and 0.25.
    - ent_coef (float): Entropy coefficient to encourage exploration, sampled logarithmically between 0.001 and 0.2.
    - vf_coef (float): Value function loss coefficient, sampled between 0.3 and 0.7.
    - max_grad_norm (float): Maximum gradient norm for clipping, sampled between 0.2 and 2.0.
    - target_kl (float): KL divergence target for stopping updates, sampled between 0.02 and 0.15.
    
    Policy Architecture:
    - activation_fn (torch.nn.Module): Activation function for neural network layers, chosen between Tanh and ReLU.
    - net_arch (List[int]): Neural network architecture, set as [64, 64] for "tiny" and [128, 128] for "small".
    """
        
    learning_rate = trial.suggest_float('learning_rate', 5e-5, 1e-3, log=True)
    n_steps = 2 ** trial.suggest_int('n_steps', 10, 12)  # Number of steps per update
    
    valid_batch_sizes = [b for b in [64, 128, 256, 512, 1024] if (n_steps * N_EVAL_ENVS) % b == 0]
    batch_size = trial.suggest_categorical('batch_size', valid_batch_sizes)
    
    n_epochs = trial.suggest_int('n_epochs', 8, 12)  # PPO update epochs per batch
    gamma = trial.suggest_float('gamma', 0.95, 0.9999)  # Discount factor (close to 1 for long-term rewards)
    gae_lambda = trial.suggest_float('gae_lambda', 0.9, 1.0)  # GAE lambda (trade-off bias/variance)
    clip_range = trial.suggest_float('clip_range', 0.15, 0.25)  # PPO clipping range
    ent_coef = trial.suggest_float('ent_coef', 0.001, 0.2, log=True)  # Entropy coefficient (for exploration)
    vf_coef = trial.suggest_float('vf_coef', 0.3, 0.7)  # Value function loss coefficient
    max_grad_norm = trial.suggest_float('max_grad_norm', 0.2, 2.0)  # Gradient clipping
    target_kl = trial.suggest_float('target_kl', 0.02, 0.15)  # KL divergence target
    
    activation_fn = trial.suggest_categorical('activation_fn', ['tanh', 'relu'])
    net_arch = trial.suggest_categorical('net_arch', ['tiny', 'small'])
    # Convert architecture choices
    net_arch = [64, 64] if net_arch == 'tiny' else [128, 128]
    activation_fn = {'tanh': nn.Tanh, 'relu': nn.ReLU}[activation_fn]
    
    return {
        'learning_rate': learning_rate,
        'n_steps': n_steps,
        'batch_size': batch_size,
        'n_epochs': n_epochs,
        'gamma': gamma, 
        'gae_lambda': gae_lambda,
        'clip_range': clip_range,
        'ent_coef': ent_coef,
        'vf_coef': vf_coef, 
        'max_grad_norm': max_grad_norm,
        'target_kl': target_kl,
        'policy_kwargs': {
            'net_arch': net_arch,
            'activation_fn': activation_fn
        }
    }

In [16]:
class TrialEvalCallback(EvalCallback):
    """
    A custom Optuna evaluation callback for tracking and pruning PPO training trials.

    This callback extends `EvalCallback` to integrate Optuna's pruning mechanism, allowing
    for the early stopping of underperforming trials based on evaluation results.

    Parameters:
    - eval_env (gym.Env): The evaluation environment used to assess the agent's performance.
    - trial (optuna.Trial): The Optuna trial object used for logging results and deciding pruning.
    - n_eval_episodes (int, optional): Number of episodes to evaluate the policy. Default is 5.
    - eval_freq (int, optional): Frequency (in environment steps) at which the policy is evaluated. Default is 10,000.
    - deterministic (bool, optional): Whether to use a deterministic policy during evaluation. Default is True.
    - verbose (int, optional): Verbosity level (0: silent, 1: info). Default is 0.

    Attributes:
    - trial (optuna.Trial): The associated Optuna trial for hyperparameter optimization.
    - eval_idx (int): The current evaluation index, incremented after each evaluation.
    - is_pruned (bool): Flag indicating whether the trial was pruned.

    Methods:
    - _on_step() -> bool: Overrides the parent method to evaluate the agent and report performance to Optuna.
      If the trial performs poorly, it may be pruned to save computational resources.

    Returns:
    - bool: True if training should continue, False if the trial should be pruned.
    """
    
    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # Evaluate policy (done in the parent class)
            super()._on_step()
            self.eval_idx += 1
            # Send report to Optuna
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

In [17]:
def objective(trial: optuna.Trial) -> float:
    """
    Defines the objective function for Optuna hyperparameter optimization of a PPO agent.

    This function samples a set of hyperparameters, initializes the training and evaluation environments, 
    trains a PPO model, and returns the mean episodic reward as the evaluation metric.

    Parameters:
    - trial (optuna.Trial): The Optuna trial object used for sampling hyperparameters and logging results.

    Returns:
    - float: The mean episodic reward after training.

    Raises:
    - optuna.exceptions.TrialPruned: If the trial is pruned due to poor performance.

    Workflow:
    1. Copies default hyperparameters and updates them with sampled values.
    2. Configures environment parameters and initializes training and evaluation environments.
    3. Creates and trains the PPO model using the sampled hyperparameters.
    4. Evaluates the model using the `TrialEvalCallback` and records performance.
    5. Handles potential issues such as NaN values during training.
    6. If the trial is pruned, it raises `TrialPruned()`, otherwise returns the last mean reward.

    Notes:
    - The function ensures proper environment closure to prevent memory leaks.
    - If NaN values are encountered during training, the function returns NaN to indicate failure.
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    
    # Sample hyperparameters
    kwargs.update(**sample_ppo_params(trial))

    # Ensure env_kwargs is passed correctly
    env_kwargs = {
        "render_mode": "rgb_array",
        "continuous": True,
        "max_episode_steps": 1200,
        "training_stage": 2,
        "timescale": 1/3,
        "total_targets": 3,
    }

    # Create the training environment
    train_env = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS, env_kwargs=env_kwargs)

    # Create the RL model
    model = PPO(
        device='cpu', 
        verbose=0, 
        env=train_env, 
        tensorboard_log=os.path.join(log_folder, 'optimize'), 
        **kwargs
    )

    # Create evaluation environment with same parameters
    eval_envs = make_vec_env(ENV_ID, n_envs=N_EVAL_ENVS, env_kwargs=env_kwargs)

    # Create callback for evaluation
    eval_callback = TrialEvalCallback(eval_envs, trial, N_EVAL_EPISODES, EVAL_FREQ, deterministic=True, verbose=0)
    
    nan_encountered = False
    try:
        # Train the model
        model.learn(N_TIMESTEPS, callback=eval_callback, progress_bar=True)
    except AssertionError as e:
        nan_encountered = True
    finally:
        model.env.close()
        eval_envs.close()

    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

In [18]:
def export_best_parameters(best_trial_parameters: Dict, file_path: str = "best_hyperparameters.json") -> Dict:
    """
    Exports the best hyperparameters from an Optuna trial to a JSON file for later use.

    This function processes and formats the best trial parameters, ensuring compatibility for saving and reloading.
    It modifies specific parameters such as the activation function, network architecture, and `n_steps` before 
    saving them to a JSON file.

    Parameters:
    - best_trial_parameters (Dict): The dictionary containing the best hyperparameters found by Optuna.
    - file_path (str, optional): The filename for saving the best parameters. Default is "best_hyperparameters.json".

    Returns:
    - Dict: The processed dictionary of best hyperparameters, including formatted policy arguments.

    Processing Steps:
    - Converts the activation function from a PyTorch class to a string for JSON compatibility.
    - Maps the network architecture from a string key to a corresponding list of layer sizes.
    - Converts `n_steps` from an exponent value to its actual power-of-2 integer.
    - Adds `policy_kwargs` containing the processed architecture and activation function.
    - Includes the `tensorboard_log` directory for logging purposes.
    - Saves the parameters in JSON format to the specified file path inside the log directory.

    Notes:
    - The function ensures that all parameters are JSON-serializable.
    - The saved JSON file allows easy reloading of the best hyperparameters for future training.
    """
    
    import json
        
    # Convert activation function string to actual PyTorch class
    activation_fn = best_trial_parameters.pop('activation_fn')
    activation_fn = {'tanh': 'Tanh', 'relu': 'ReLU'}[activation_fn]  # Store as string for JSON compatibility

    # Convert network architecture
    net_arch = best_trial_parameters.pop('net_arch')
    net_arch = [64, 64] if net_arch == 'tiny' else [128, 128]

    # Prepare policy kwargs
    policy_kwargs = {'net_arch': net_arch, 'activation_fn': activation_fn}

    # Convert n_steps from power of 2
    best_trial_parameters['n_steps'] = 2 ** best_trial_parameters['n_steps']

    # Add policy kwargs
    best_trial_parameters['policy_kwargs'] = policy_kwargs
    # Add tensorboard logdir
    best_trial_parameters['tensorboard_log'] = log_folder
    
    # Save parameters to a JSON file
    path = os.path.join(log_folder, file_path)
    with open(path, "w") as json_file:
        json.dump(best_trial_parameters, json_file, indent=4)

    print(f"Best hyperparameters saved to {file_path}")  
    
    return best_trial_parameters

In [19]:
def load_hyperparameters(file_path: str = "best_hyperparameters.json") -> Dict:
    """
    Loads previously saved hyperparameters from a JSON file and converts the activation function 
    string back to its corresponding PyTorch class.

    Parameters:
    - file_path (str, optional): The filename from which to load hyperparameters. Default is "best_hyperparameters.json".

    Returns:
    - Dict: A dictionary containing the loaded hyperparameters with correctly formatted values.

    Processing Steps:
    - Reads the JSON file from the specified path.
    - Converts the activation function string ("Tanh" or "ReLU") back to its PyTorch equivalent (`nn.Tanh` or `nn.ReLU`).
    - Returns the fully formatted hyperparameter dictionary.

    Notes:
    - This function assumes that the JSON file was created using `export_best_parameters()`.
    - The `log_folder` must be defined as the directory where the JSON file is stored.
    - If the file does not exist or contains invalid data, an exception may be raised.
    """
    
    import json
        
    # Load JSON data
    path = os.path.join(log_folder, file_path)
    with open(path, "r") as json_file:
        hyperparams = json.load(json_file)
    
    # Convert activation function from string to PyTorch class
    activation_fn = hyperparams['policy_kwargs']['activation_fn']
    hyperparams['policy_kwargs']['activation_fn'] = {'Tanh': nn.Tanh, 'ReLU': nn.ReLU}[activation_fn]

    print(f"Hyperparameters loaded from {file_path}")

    return hyperparams

### 5.3 Optimization Workflow
#### 5.3.1 Trial Evaluation with Callbacks
Each trial trained a PPO agent using the sampled hyperparameters and evaluated it periodically using a custom TrialEvalCallback. The callback:

- Monitored performance after every 10,000 training steps.
- Logged evaluation metrics to Optuna.
- Pruned underperforming trials to save computation time. This ensured that only promising hyperparameter configurations were fully trained.

#### 5.3.2 Environment Setup for Training & Evaluation
To ensure realistic performance evaluation, the environment was initialized with:

- Waypoint tracking enabled
- Three dynamic targets in head-on, crossing, or static situations
- Timescale set to 1/3, meaning each step represented 20 seconds of real-world time
- A vectorized environment (n_envs=10) was used for efficient parallel training and evaluation.

#### 5.3.3 Hyperparameter Optimization Strategy
To efficiently search for the best hyperparameter configurations, Optuna used:

- MedianPruner: This pruning strategy stops underperforming trials early based on their intermediate performance relative to the median of all completed trials. It helps allocate resources efficiently by discarding trials unlikely to yield improvements.
- TPE Sampler: The Tree-structured Parzen Estimator (TPE) is a Bayesian optimization algorithm that models promising hyperparameter regions based on past trials. Instead of random sampling, it selects hyperparameters more likely to improve performance, leading to faster convergence.


By combining pruning and adaptive sampling, the optimization process efficiently explored the hyperparameter space while focusing computational resources on the most promising configurations.

In [20]:
# from optuna.pruners import MedianPruner
# from optuna.samplers import TPESampler

# # Set pytorch num threads to 1 for faster training
# torch.set_num_threads(1)
# # Select the sampler, can be random, TPESampler, CMAES, ...
# sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# # Do not prune before 1/3 of the max budget is used
# pruner = MedianPruner(
#     n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 2
# )
# # Create the study and start the hyperparameter optimization
# study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

# try:
#     study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT, show_progress_bar=True)
# except KeyboardInterrupt:
#     pass

# print("Number of finished trials: ", len(study.trials))

# print("Best trial:")
# trial = study.best_trial

# print(f"  Value: {trial.value}")

# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

# print("  User attrs:")
# for key, value in trial.user_attrs.items():
#     print(f"    {key}: {value}")

# # Write report
# study.trials_dataframe().to_csv(os.path.join(log_folder, "study_results_ppo_marineenv.csv"), index=False)

### 5.4 Results & Best Hyperparameters
After 100 trials with 200 000 timesteps each, the best performing PPO configuration was:

In [21]:
# best_kwargs = export_best_parameters(study.best_params)

In [22]:
best_kwargs = load_hyperparameters()

Hyperparameters loaded from best_hyperparameters.json


In [23]:
best_kwargs

{'learning_rate': 0.00022065441901241827,
 'n_steps': 2048,
 'batch_size': 256,
 'n_epochs': 8,
 'gamma': 0.9908682940291461,
 'gae_lambda': 0.914493830486739,
 'clip_range': 0.20130150169819302,
 'ent_coef': 0.0023625133272582688,
 'vf_coef': 0.43442214543301433,
 'max_grad_norm': 0.7865302683535346,
 'target_kl': 0.04389071049636425,
 'policy_kwargs': {'net_arch': [64, 64],
  'activation_fn': torch.nn.modules.activation.Tanh},
 'tensorboard_log': './project_logs'}

### 5.5 Training the agent with the optimized hyperparameters

In [24]:
# Create the evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=os.path.join(log_folder, 'optuna/best_model/'),
    log_path=os.path.join(log_folder, 'optuna/results/'),
    eval_freq=5000,
    deterministic=True,
    render=False
)

In [25]:
optuna_agent = PPO(
    policy='MlpPolicy',
    env=train_env,
    verbose=0,
    device='cpu',
    **best_kwargs,
)

In [26]:
# optuna_agent.learn(total_timesteps=(6e5), reset_num_timesteps=True, progress_bar=True, tb_log_name='optuna/optuna_best', callback=eval_callback)

Below comparing of the agents using fixed random seed

In [27]:
seed = next(seed_generator)
env_kwargs = dict(
    render_mode="rgb_array",
    continuous=True,
    training_stage=2,
    timescale=1/3,
    training=False,
    total_targets=3,
    seed=seed,
)

# Load the default-trained model
default_model = PPO.load(os.path.join(log_folder, 'default/best_model/best_model.zip'), device='cpu')
default_env = Monitor(gym.make("MarineEnv-v0", **env_kwargs))

# Load the optimized model
optimized_model = PPO.load(os.path.join(log_folder, 'optuna/best_model/best_model.zip'), device='cpu')
optimized_env = Monitor(gym.make("MarineEnv-v0", **env_kwargs))

# Evaluate both models
mean_default, std_default = evaluate_policy(default_model, default_env, n_eval_episodes=10, deterministic=True)
mean_optimized, std_optimized = evaluate_policy(optimized_model, optimized_env, n_eval_episodes=10, deterministic=True)

print(f"Default Model: Mean Reward = {mean_default:.2f}, Std = {std_default:.2f}")
print(f"Optimized Model: Mean Reward = {mean_optimized:.2f}, Std = {std_optimized:.2f}")

Seed set to 83
Seed set to 83
Default Model: Mean Reward = 458.91, Std = 445.97
Optimized Model: Mean Reward = 736.06, Std = 155.27


In [28]:
# record video
# run_video_rendering(optimized_model, episodes=10, seed=185, record_video=True, video_folder=video_folder, name_prefix='optimized_agent')

In [29]:
# run_video_rendering(default_model, seed=25, episodes=10)

### 5.6 Model Evaluation
Hyperparameter tuning with Optuna was used to maximize the ASV’s waypoint tracking efficiency and COLREG compliance.

TPESampler & Median Pruning helped efficiently search for high-performing configurations.

Custom Trial Evaluation Callback ensured continuous assessment and pruning of poor trials.

Best PPO configuration was identified after 100 trials, with somehow questionable improvements in navigation and decision-making.

The optimized model was then used for final training and deployment, where it consistently navigated toward waypoints while complying with maritime regulations similar to the agent, trained with defaut hyperparameters of the SB3 PPO algorithm.

The study results are saved as `csv`, allowing to examine the conducted trials at later stage.

**Reading of the training statistics**

1. FPS (Frames Per Second) measures the training speed of the agent, indicating how efficiently the environment steps are processed. The optimized model (Optuna) maintains a consistently higher FPS, starting at 4355 and stabilizing around 3490, while the default model starts at 4041 but drops to 2074. This suggests that the optimized hyperparameters improve computational efficiency and training speed.

2. Entropy Loss measures the randomness of the agent’s policy; closing to zero indicates less exploration and more deterministic actions. Optimized agent maintaines more negative entropy compared to defaut agent, suggesting better exploration before settling into a more stable policy.

3. Explained Variance measures how well the value function predicts returns, with values closer to 1 indicating better predictions.

Default model ranges from -4.41e-06 to 0.794, showing strong improvement while optimized model ranges from 0.00053 to 0.741, slightly lower max but more stable.
Both models improve prediction quality, but the default model reaches a higher peak accuracy, while the optimized model may be more stable in variance estimation considering the fluctuation during training.
Deafut model start with negative value, which could in suggest high initial bias, however it improved and the variance stabalized, which could indicate less bias and controlled variance. Same applies for the optimized model as well but showing better balance.

4. Total Loss represents the overall objective function value, including policy and value losses. Default model ranges from 8984.49 ot 118.15, indicating larger fluctuations compared to optimized model with ranges from 3072.49 to 181.71, with significantly lower peak loss.
The optimized model has lower total loss and reduced variability, suggesting a more stable and efficient training process.

5. Policy Gradient Loss measures how much the policy is changing; more negative values indicate stronger updates. The default model ranges from -0.0068 to -0.0016, while the optimized model ranges from -0.0075 to -0.0026, showing consistently stronger updates. Combined with explained variance, this could mean that the optimized model has lower bias and more controlled updates, leading to a better balance between learning stability.

6. Value Loss measures how well the value function approximates expected returns, with lower values indicating better learning stability. The default model has a high range (735.14 to 18,368.38), showing large fluctuations, while the optimized model has a significantly lower range (574.97 to 8,906.79), suggesting more stable learning.

7. Approximate KL Divergence measures how much the new policy deviates from the old one, with lower values indicating more stable updates. The default model ranges from 0.0042 to 0.0079, while the optimized model has a slightly lower range 0.0039 to 0.0062, suggesting more conservative policy updates. Thus the optimized model has smoother policy updates, reducing drastic shifts and ensuring more stable learning.

8. Clip Fraction represents the proportion of policy updates that exceed the clipping range in PPO, with lower values indicating more stable updates. The default model ranges from 0.0295 to 0.0796, while the optimized model has a lower range (0.0268 to 0.0576), suggesting that the optimized hyperparameters lead to smoother, more controlled updates, reducing excessive policy shifts and improving training stability

9. Reward measures the agent's performance, with higher values indicating better policy effectiveness. The default model ranges from -1043.26 to 566.54, while the optimized model achieves a slightly better range of -1039.95 to 594.79, indicating improved learning and higher peak rewards. This suggests that the optimized hyperparameters lead to better overall performance and more efficient policy optimization.

### 5.7 Optimization summary
The provided video files of both the default and optimized models serve as a visual aid for evaluating the agents' performance. In summary, both models adhere to COLREGs without significant violations. However, in episode 3, the default model fails to reach the waypoint, whereas the optimized model successfully completes the episode.

## 6. Follow-Up Work and Future Improvements
The current implementation demonstrates a partially COLREG-compliant ASV trained with Deep Reinforcement Learning (DRL) to navigate towards a waypoint while avoiding collisions. However, there are several areas for future work to improve realism, policy effectiveness, and robustness in real-world deployment.

### 6.1 Extending COLREG Compliance
Currently, the agent follows COLREGs for head-on and crossing situations, but the overtaking rule (Rule 13) has been omitted.

**Next step**: Implement full COLREG compliance, including overtaking scenarios where:

The ASV recognizes when it is overtaking or being overtaken.
The agent adjusts its maneuvers accordingly, ensuring it passes safely.

### 6.2 Multi-Agent Reinforcement Learning (MARL)
The current approach assumes static or scripted target ship behaviors. However, in reality, vessels react dynamically to COLREG rules.

**Next step**: Implement a multi-agent environment where:

- Target ships are also reinforcement learning agents, pre-trained on giving way.
- The own vessel is trained to "stand on" but also learns last-minute evasive actions if the target fails to give way.

This setup creates a more realistic simulation, increasing the robustness of the ASV’s policy.

### 6.3 Evaluating Different RL Algorithms
PPO has been chosen for its stability, but alternative RL algorithms could be tested for better performance:

**Next step**: Compare performance of:

- Soft Actor-Critic (SAC) → More sample efficient, learns robust policies.
- Twin Delayed DDPG (TD3) → Handles continuous actions well, reduces overestimation bias.
- Hierarchical RL (HRL) → Train separate policies for waypoint tracking and collision avoidance, then combine them.

### 6.4 Incorporating Environment and Ship Dynamics
Currently, the environment ignores realistic ship dynamics, assuming an instant response to actions.

**Next step**: Add real-world ship dynamics to the environment:

- Wind and current effects: The ASV should learn how to compensate for external forces.
- Turning rate constraints: The ship's ability to turn depends on speed and rudder angles.
- Stopping dynamics: The ASV should account for braking distance.
- Engine delay: Speed changes take time to apply, affecting maneuvering.

These changes would increase realism but also make learning harder, requiring smarter training strategies.

### 6.5 Scaling Up: Larger Environments and Complex Passage Plans
Current training involves a single waypoint and a small area.

**Next step**:

- Train in larger environments with multiple waypoints to simulate full passage plans.
- Introduce randomized targets along the route, making the ASV handle longer-term decision-making.
- Measure policy effectiveness over long distances and evaluate navigational efficiency.

### 6.6 Using Real-World AIS Data for Training
The current model relies on synthetic target behaviors, but real-world ship maneuvering follows complex, human-driven decision-making patterns.

**Next step**:

- Obtain real-world AIS (Automatic Identification System) data from ships.
- Train the RL model on real ship behaviors, improving its ability to predict maneuvers.
- Test the DRL model in actual maritime environments to see how well it generalizes.

### 6.7 Hardware Deployment and Real-World Testing
The model is trained in simulation, but real-world deployment requires:

**Next step**:

- Deploy the trained model onto a real ASV or high-fidelity maritime simulator.
- Test how well the RL policy generalizes to real-world conditions.
- Implement sensor-based perception (radar, cameras, AIS) instead of synthetic inputs.
### 6.8 Funding and Resource Considerations
To develop high-fidelity, real-world RL-driven ASVs, significant funding would be required for:

- High-performance computing resources for large-scale DRL training.
- Access to AIS and real ship maneuver data.
- Developing real-world test environments, including physical ASVs for deployment.
- Collaboration with maritime institutions for regulatory compliance and testing.

Potential funding sources include:

- Government maritime research grants.
- Private sector funding from shipping and AI companies.
- Collaborations with academia and navies.

## 7. Conclusion
The developed PPO-based autonomous navigation model demonstrates high performance in the given environment, efficiently following COLREGs while maintaining an optimal route to the waypoint. The agent successfully balances compliance with collision avoidance rules and ETA accuracy, adjusting its speed only when necessary to avoid excessive deviations.

One of the key strengths of the model is its scalability. Since the observation space is based on relative bearings and distances rather than absolute latitude/longitude coordinates, the model can be easily extended to handle more dynamic targets without requiring major architectural changes. This flexibility makes it well-suited for real-world maritime navigation scenarios involving multiple vessels.

Despite its strong performance, reward function optimization remains an area for improvement. Refining the reward structure could further enhance decision-making efficiency, particularly in scenarios requiring complex maneuvers or long-term planning. Additionally, adaptive exploration strategies could help the model generalize better to varying traffic densities and environmental conditions.

Overall, this study demonstrates the viability of reinforcement learning for autonomous marine navigation, providing a robust foundation for future enhancements such as multi-agent coordination, dynamic weather adaptation, and real-world sensor integration.

## 8. References
In Chapter 2: Previous Research, a review of relevant academic articles has been provided and will not be listed here. Below are the primary sources and tools referenced throughout this project:

- International Maritime Organization (IMO) & COLREGs – The International Regulations for Preventing Collisions at Sea, which define the legal framework for vessel navigation and avoidance maneuvers.
- PyTorch – Open-source deep learning framework used for implementing and training the reinforcement learning model.
- Hugging Face – Provides tools and model repositories for reinforcement learning and deep learning applications.
- Stable-Baselines3 (SB3) – A reliable reinforcement learning library built on PyTorch, used for training the PPO agent.
- Optuna – A hyperparameter optimization framework that was utilized for fine-tuning the reinforcement learning model, improving efficiency and performance.
- Gymnasium (formerly OpenAI Gym) – A toolkit for developing and comparing reinforcement learning algorithms, used to create and test the training environment.
- OpenAI – Provided foundational research and tools in deep reinforcement learning, contributing to the development of stable baselines and policy optimization techniques.
- ChatGPT – Assisted in structuring explanations, refining technical documentation, and improving clarity in methodology and findings.
  
These sources have been instrumental in the development and implementation of the model, ensuring both compliance with maritime navigation rules and best practices in reinforcement learning.